In [ ]:
# import packages
import pandas as pd
from datetime import datetime

import ast

import matplotlib.pyplot as plt

In [ ]:
# read the datasets
movies_metadata = pd.read_csv('movies_metadata.csv')
ratings = pd.read_csv('ratings.csv')
links = pd.read_csv('links.csv')
keywords = pd.read_csv('keywords.csv')
credits = pd.read_csv('credits.csv')

ratings['date_time'] = pd.to_datetime(ratings['timestamp'], unit='s')

Genre

In [ ]:
# print the count of genres
movies_metadata['genres_list'] = movies_metadata['genres'].apply(ast.literal_eval)

all_genres = dict()
for genre_list in movies_metadata['genres_list']:
    for genre in genre_list:
        if genre['name'] not in all_genres.keys():
            all_genres[genre['name']] = 1
        else:
            all_genres[genre['name']] += 1

print(f"Total unique genres: {len(all_genres)}")
print(all_genres)

In [ ]:
# print one-time genre
onetime_genres = []

for genre in all_genres.keys():
    if all_genres[genre] == 1:
        onetime_genres.append(genre)
  
print(onetime_genres)

In [ ]:
# get the dummy variables for genres
for genre in all_genres.keys():
    movies_metadata[f"genre_{genre}"] = movies_metadata['genres_list'].apply(
        lambda x: int(any(g['name'] == genre for g in x))
    )
   
# remove the one-time genre dummy variable
movies_metadata = movies_metadata.drop([f"genre_{genre}" for genre in onetime_genres], axis = 1)

In [ ]:
# get the genre columns
genre_columns = [col for col in movies_metadata.columns if col.startswith('genre_')]

# get the genre average votes dictionary
genre_avg_votes = {
    genre.split('_')[1]: movies_metadata[movies_metadata[genre] == 1]['vote_average'].mean()
    for genre in genre_columns
}

# change the dictionary to dataframe
genre_avg_df = pd.DataFrame.from_dict(genre_avg_votes, orient='index', columns=['avg_vote'])
genre_avg_df = genre_avg_df.sort_values(by='avg_vote', ascending=False)

# graph
genre_avg_df.plot(kind='barh', figsize=(10, 6), legend=False)
plt.title('Average Vote by Genre')
plt.xlabel('Average Vote')
plt.ylabel('Genre')
plt.tight_layout()
plt.show()

Release Year

In [ ]:
movies_metadata['release_date'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce')
movies_metadata['release_year'] = movies_metadata['release_date'].dt.year

In [ ]:
year_count_df = pd.DataFrame.from_dict(dict(movies_metadata.groupby('release_year').size()), orient='index', columns=['count'])

# graph
year_count_df.plot(kind='line', figsize=(10, 6), legend=False)
plt.title('Movie Count by Year')
plt.xlabel('Year')
plt.ylabel('Movie Count')
plt.tight_layout()
plt.show()